<a href="https://colab.research.google.com/github/EricW1118/NLP_Res/blob/main/LSTM_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libs needed
!pip3 install torch 
!pip3 install torchvision
!pip3 install torchtext
!pip3 install pandas
!pip3 install nltk
!pip3 install -U scikit-learn scipy matplotlib
!pip3 install spacy
!pip3 install transformers
!pip3 install seaborn

In [ ]:
# Import libs
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import re
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import string
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Download wordnet for lemmatization
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Download the stopwords from nltk
nltk.download('stopwords')
ntst = stopwords.words('english')
print(ntst)
print(len(ntst))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Import stopwords from spacy
from spacy.lang.en.stop_words import STOP_WORDS
print(STOP_WORDS)
print(len(STOP_WORDS))

{'well', 'it', 'top', 'thereupon', 'see', 'either', 'sometimes', 'thereby', 'not', 'in', 'than', 'using', 'much', 'without', 'less', 'during', 'rather', "n't", 'herself', 'once', 'been', '’d', 'who', 'however', 'mine', 'about', 'get', 'meanwhile', 'becomes', 'three', 'sometime', 'those', 'now', 'down', 'nowhere', 'be', 'or', 'himself', '‘d', 'beyond', 'of', 'throughout', 'make', 'still', 'some', 'being', 'per', 'although', 'namely', 'was', '‘s', 'itself', 'over', 'few', 'all', 'besides', 'last', 'wherein', 'these', '’ll', 'back', 'ten', 'latterly', 'otherwise', 'with', 'elsewhere', 'had', 'almost', 'empty', 'but', 'really', 'own', 'above', 'thereafter', 'by', 'somehow', 'yours', 'ever', "'d", 'hereupon', 'thence', 'since', 'has', 'every', 'before', 'whose', 'give', 'nevertheless', 'what', 'most', '‘m', 'beside', 'did', 'moreover', 'eleven', 'only', 'amongst', 'often', 'me', 'fifteen', 'fifty', 'many', 'does', 'hence', 'here', 'each', 'off', 'hereby', 'out', 'mostly', 'always', 'also', 

In [ ]:
# Merge stopwords from two libs
stwords = set(ntst).union(STOP_WORDS)
print(stwords)
print(len(stwords))

{'well', 'it', 'top', 'he', 'thereupon', 'see', 'us', 'either', 'sometimes', 'amount', 'thereby', 'latter', 'together', 'haven', 'such', 'onto', "hadn't", 'while', 'in', 'not', 'than', 'using', 'cannot', 'the', 'much', 'though', 'without', 'very', 'less', "hasn't", 'during', 'rather', "n't", 'through', 'herself', 'when', 'once', 'whole', 'whereupon', 'put', 'been', '’d', 'who', 'which', "mightn't", 'so', 'however', 'mine', 'about', 'get', 'whatever', 'hasn', 'therein', 'meanwhile', 'becomes', 'three', 'seems', 'just', 'ca', 'sometime', 'your', 'those', 'one', 'now', 'down', 't', 'around', "couldn't", 'nowhere', 'hers', '‘ve', 'be', 'or', 'then', 'myself', 'himself', 'theirs', '’re', 'beforehand', '‘d', "you'll", 'of', 'beyond', 'four', 'throughout', 'full', 'anywhere', 'make', 'hereafter', "you're", 'against', 'still', 'some', "'m", 'being', 'per', 'whereafter', 'anyone', 'an', 'thru', 'although', 'namely', 'was', '‘s', 'itself', 'toward', 'whither', 'mustn', 'no', 'why', 'via', 'over'

In [ ]:
# Delete the stopwords
def deleteStopwords(row):
    text = str(row['text'])
    tx_arr = text.split()
    row['text'] = ' '.join( [w for w in tx_arr if w not in stwords])
    return row

# Remove punctuations including Chinses and English styles
def remove_punctuation(row):
    row['text'] = str(row['text']).translate(str.maketrans('', '', string.punctuation))
    # Chinese punctuation
    pattern = r'[。，“”‘’！？：；【】《》「」『』（）［］〔〕]'
    # Use re.sub() to replace all matches with an empty string
    row['text'] = re.sub(pattern, '', row['text'])
    return row

# Apply Stemming operation for each row
stemmer = PorterStemmer()
def stemming(row):
    text = str(row['text'])
    tx_arr = text.split()
    row['text'] = ' '.join([stemmer.stem(w) for w in tx_arr])
    return row

# Apply lemmatization operation for each row
lemmatizer = WordNetLemmatizer()
def lemmatize(row):
    text = str(row['text'])
    tx_arr = text.split()
    row['text'] = ' '.join([lemmatizer.lemmatize(w) for w in tx_arr])
    return row

# After stopwords and puntuations removal, there may be some empty row in text, replace this row with np.naN
# Then we can apply dropna to eliminate these rows
def removeEmpty(row):
    text = str(row['text'])
    if text.isspace() or len(text) <= 0:
        row['text'] = np.NaN
    return row
  
# Load the csv from the path given in the parameter
# Apply different processing methods defined above
def load_csv(file_path):
    df = pd.read_csv(file_path)
    df.columns = ["id","title","text","label"]
    df.drop(labels=['id','title'], axis='columns', inplace=True)
    df['text'] = df['text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
    df = df.apply(remove_punctuation, axis=1)
    df = df.apply(stemming, axis=1)
    df = df.apply(lemmatize, axis=1)
    df = df.apply(deleteStopwords, axis=1)
    df = df.apply(removeEmpty, axis=1)
    df = df.dropna(subset=['text'], axis=0, how= 'all')
    return df

In [ ]:
# Only if you are using Google Colab, you need to mount the Google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Use the function defined above to load the csv data
# df = load_csv('WELFake_Dataset.csv')
df = load_csv('/content/gdrive/My Drive/NLP/WELFake_Dataset.csv')

In [ ]:
# Print the data summary 
df.info()

# Print some rows of the dataset
print(df)

In [ ]:
# Check if the data is balanced between all the classes
data_count = df['label'].value_counts()
sns.barplot(x=np.array([0,1]),y=data_count.values)
plt.show()

In [ ]:
# Plot the length distribution of sentences of the dataset
rev_len = [len(i) for i in  df['text']]
pd.Series(rev_len).hist()
plt.show()

# Summarize the sentence length distribution
pd.Series(rev_len).describe()

In [ ]:
# Calculate the number of rows with lable 1 and 0
df_zero = df[df['label']==0]
df_one = df[df['label']==1]

print('length of sentences with label zero : %4d' % (len(df_zero)))
print('length of sentences with label one: %4d' % (len(df_one)))

In [ ]:
# Calculate 80% of the number of class 0 and class 1
# Then take the smaller one 
train_length = int(min(len(df_zero) * 0.8, len(df_one) * 0.8))
print('length of each train set: %4d ' % (train_length))

# Randomly pick the number calculated above of rows from class 1 and class 0
# This operation makes sure we take the same number of rows from each class
# Make sure our training dataset is balanced
zero_train, zero_test = train_test_split(df_zero, train_size=train_length, shuffle=True, random_state=36)
one_train, one_test = train_test_split(df_one,  train_size=train_length, shuffle=True, random_state=36)

# Concate the rows selected above from each label to form the final training set
train, test = pd.concat([zero_train, one_train]), pd.concat([zero_test, one_test])
print('length of final train set: {}'.format(len(train)))
print('length of final test set: {}'.format(len(test)))

In [ ]:
def tockenize(x_train, y_train, x_val, y_val):
    word_list = []
    for sent in x_train:
      for word in sent.split():
        word_list.append(word)

    corpus = Counter(word_list)
    print(len(corpus))
    # sorting on the basis of most common words
    corpus_sorted = sorted(corpus, key=corpus.get, reverse=True)[:20000]
    # creating a dict
    onehot_dict = {w:i+1 for i,w in enumerate(corpus_sorted)}
    # tockenize
    final_list_train, final_list_test = [],[]

    for sent in x_train:
            final_list_train.append([onehot_dict[word] for word in sent.split() 
                                     if word in onehot_dict.keys()])
    for sent in x_val:
            final_list_test.append([onehot_dict[word] for word in sent.split() 
                                    if word in onehot_dict.keys()])

    # encoded_train = [label for label in y_train]  
    # encoded_test = [label for label in y_val] 
    
    # print(final_list_train)
    # return np.array(final_list_train), np.array(y_train), np.array(final_list_test), np.array(y_val), onehot_dict
    return final_list_train, y_train, final_list_test, y_val, onehot_dict

In [ ]:
# Use the method definve above to do the tokenization for training and test dataset
x_train, y_train, x_test, y_test, vocab = tockenize(train['text'], train['label'], test['text'], test['label'])

In [ ]:
print(f'Length of vocabulary is {len(vocab)}')

In [ ]:
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [ ]:
x_train_pad = padding_(x_train,800)
x_test_pad = padding_(x_test,800)

In [ ]:
print(x_train_pad)
print(len(x_train_pad))

print(x_test_pad)
print(len(x_test_pad))

In [ ]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(np.array(x_train_pad)), torch.from_numpy(np.array(y_train)))
valid_data = TensorDataset(torch.from_numpy(np.array(x_test_pad)), torch.from_numpy(np.array(y_test)))

# dataloaders 
batch_size = 100

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)

In [ ]:
# is_cuda = not torch.cuda.is_available
is_cuda = torch.cuda.is_available
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, CPU used")

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        embedded = self.embedding(x)
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(embedded, (h0, c0))

        # Attention mechanism
        attention_weights = torch.softmax(self.attention(out), dim=1)
        attention_out = torch.sum(attention_weights * out, dim=1)
        
        # Extract the last timestep's output
        # out = out[:, -1, :]

        # out = self.fc(out)
        out = self.fc(attention_out)
        return out


In [ ]:
# Define the model
embedding_dim = 80
num_classes = 2
vocab_size = len(vocab) + 1 #extra 1 for padding
hidden_size = 100
num_layers = 12
num_classes = 2

# Create the model
model = LSTMClassifier(vocab_size, embedding_dim, hidden_size, num_layers, num_classes)
model.to(device)

In [ ]:
# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

In [ ]:
import torch.optim as optim

# Given labels and predictions accordingly, compute the number of the right prediction
def num_of_acc(pred,label):
    _, predicted = torch.max(pred, dim=1)
    # print(predicted.shape, label.shape)
    # pred = torch.round(pred.squeeze())
    # return torch.sum(pred == label.squeeze()).item()
    return torch.sum(predicted == label.squeeze()).item()

# Given an model and dataloader, compute the accuracy
def acc_of(model,dataloader):
    model.eval()
    num_sample = 0
    accnum = 0
    for X, y in dataloader:
      num_sample += len(X)
      X, y = X.to(device), y.to(device)
      # print(X.shape, y.shape)
      outputs = model(X)
      batch_acc = num_of_acc(outputs,y)
      accnum += batch_acc
    return (accnum / num_sample)

criterion = nn.CrossEntropyLoss()  # Assuming your classification task has multiple classes
# criterion = nn.MultiMarginLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.00003)  # Adjust the learning rate as needed
num_epochs = 50  # Define the number of training epochs
optimizer = optim.NAdam(model.parameters(), lr=0.001) 

for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    train_acc = 0.0
    num_sample = 0
    for i ,(X, y) in enumerate(train_loader):
        num_sample += len(X)
        X, y = X.to(device), y.to(device)
        # Zero the gradients
        optimizer.zero_grad()
        # Perform forward pass
        outputs = model(X)
        # print(outputs.shape, y.shape)
        batch_acc = num_of_acc(outputs,y)
        train_acc += batch_acc
        loss = criterion(outputs, y)
        # Perform backward pass
        loss.backward()
        optimizer.step()
    # Print the loss for monitoring the training progress
    val_acc = acc_of(model, valid_loader)
    print('Epoch: %5d, Loss: %.5f, train_acc: %.5f, val_acc: %.5f' % (epoch + 1, loss.item(), train_acc / num_sample, val_acc))

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score
def plot_auc_roc(model,valid_loader, version='title', threshold=0.5):
    y_pred = []
    y_true = []
    model.eval()
    with torch.no_grad():
      for inputs, labels in valid_loader :
            inputs = inputs.to(device)
            output = model(inputs)

            output = (output > threshold).int()
            _, output = torch.max(output, dim = 1)
            # print(output)
            y_pred.extend(output.tolist())
            y_true.extend(labels.tolist())
    
    fpr, tpr, threshold = metrics.roc_curve(y_true, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    
    print('AUC ROC : %f ' % (roc_auc))
    print('----------------------------------------------------------')
    
    plt.title('AUC ROC Curve')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
  
plot_auc_roc(model, train_loader) 

plot_auc_roc(model, valid_loader) 